# Pogramming the Experiment
In this section we will learn how we can complete the code outline from the previous section to form a coherent psychology experiment, namely a simplified **Stroop Experiment**. We will go through the code step-by-step and at the end all these elements that we will illustrate using code snippets will then form an entire python script that forms our experiment. So this section is organized as follows:
- Importing Modules and Creating a run_experiment() Function
- Asking for Demographics
- Initializing PyGame and Experiment
- Specifying a quit_pygame() Function
- Loading Stimuli
- Creating Instruction Presentation Functions
- Implementing Stroop Task
- Saving Results
- Complete Stroop Experiment

---

## Importing Modules and Creating a `run_experiment()` Function

When considering the **code outline** of the previous section, the first thing we need to do in our main python script (`stroop.py`) is to import all modules that we want to use. This includes modules that are already pre-built and modules that we have written ourselves (e.g. `TextPresenter.py`, `config.py`):
```python
# === import modules === #
import pygame
from datetime import datetime
import os
import sys
import csv
import random
from itertools import zip_longest
# === import custom modules === #
import TextPresenter
from config import *
```
Note that the **asterix (*)** in the statement `from config import *` means that everything from that module should be imported.<br><br>
We can also define our `run_experiment()` function. In this function we will call functions that do not exist yet and our job will be to implement them. For instructional purposes we will provide all these helper functions as if we know beforehand how many we needed exacrtly and implement them below, but please note that when you are programming your own experiment, adding functions to the `run_experiment()` function is more of an iterative process, as the experiment develops. So in the following we provid the final `run_experiment()` function, but keep in mind that this is the final result and usally is not defined in advance entirely, but rather iteratively (e.g. first the `demographics_input()` function is implemented and tested, then the `init_pygame_and_exp()` function etc.). Here is the entire `run_experiment()` function:
```python
# ===  define procedure that runs the experiment === #
def run_experiment():
    """runs the experiment."""

    # ask for demographics
    demographics_input()

    # initialize pygame and font
    init_pygame_and_exp()

    # load stimuli
    load_stimuli()

    # start welcome, inst1 and inst 2 block
    start_welcome_block()
    start_inst1_block()
    start_inst2_block()
    # start stroop task
    start_begintask_block()
    start_task()
    start_endtask_block(1.0)

    # debriefing and endtask
    start_goodbye_block()

    # save results to file
    save_results(settings["filename"], results)

    # exit experiment
    quit_pygame()
```


## Asking for Demographics

Next, we will implement a function that collects demographic information from the participant and stores it in the `results` dictionary:
```python
def demographics_input():
    """Asks for participant demographics."""

    results["id"].append(input("Please enter an ID: "))
    results["age"].append(input("Please enter your age: "))
    results["gender"].append(input("Please enter your gender (m/f/other): "))
    results["major"].append(input("Please enter your major: "))
```
Note how we use the `append` method as the values of the `results` dictionary are lists. We do this in order to easier save the contents to a csv results file later using the `zip_longest` method from the python module `itertools`. So do not worry about the fact that all the entries of the dictionary are lists. We will explain this later when we implement the function that saves the results to file.

## Initializing PyGame and Experiment

Once the demographics are collected from the command line, we want to initialize PyGame together with all of our experiment settings. This is done in the function `init_pygame_and_exp()`. Here is the entire function:
```python
def init_pygame_and_exp():
    """
    initializes pygame backends explicitly with
    predefined settings.
    """

    # initialize pygame modules
    pygame.init()

    # define results filname
    settings["filename"] = results["id"][0] + \
                          "_stroop_data_" + \
                          datetime.now().strftime('%Y-%m-%d-%H-%M-%S') + \
                          '.csv'

    # get all relevant paths
    settings["absPath"] = os.path.abspath(os.curdir)
    settings["instPath"] = os.path.join(settings["absPath"], "instructions")
    settings["stimuliPath"] = os.path.join(settings["absPath"], "stimuli")
    settings["dataPath"] = os.path.join(settings["absPath"], "data")

    # load all instructions
    instructions["welcome"] = load_instructions("welcome.txt")
    instructions["intro1"] = load_instructions("intro1.txt")
    instructions["intro2"] = load_instructions("intro2.txt")
    instructions["startTask"] = load_instructions("starttask.txt")
    instructions["endTask"] = load_instructions("endtask.txt")
    instructions["goodbye"] = load_instructions("goodbye.txt")

    # define screen settings
    settings["screen"] = pygame.display.set_mode(settings["screenSize"], pygame.FULLSCREEN)
    pygame.mouse.set_visible(False) # disable mouse

    # set frame rate
    clock = pygame.time.Clock()
    clock.tick(settings["FPS"])

    # get screen rect and set font
    settings["instFont"] = pygame.font.SysFont("Arial", 30)
    settings["itemFont"] = pygame.font.SysFont("Arial", 40)
    settings["screenRect"] = settings["screen"].get_rect()

    # set instruciton text width and height
    settings["instWidth"] = settings["screenSize"][0] - (settings["screenSize"][0] // 10)
    settings["instHeight"] = settings["screenSize"][1] - (settings["screenSize"][1] // 10)
```
1. In the first part we initialize PyGame and define a filename in which we include tha current date and time using the `datetime` module. Then we get all paths to directories that we want to acess during the experiement (e.g. instructions, data, etc.). Herby, we use `os.path.join()` to join the absolute path with a path of interest (e.g. instructions). It is important to use this method, because hard coding regular slashes as either "/" or "\" for the directory paths can be error prone: Some operating systems use a backslash "\" and others a forward slash "\" and applying the `join()` method takes care of this internally, depending on the operating system our program runs on.
<br><br>
2. Next, we load all of our instructions into the `instructions` dictionary. This is done using another helper function named `load_instructions()`. The function needs to be implemented somewhere in the script and looks like this:
```python
def load_instructions(filename):
    """
    loads instructions from a text file.
    arg: name of file
    returns: content of file
    """

    # open file
    with open(os.path.join(settings["instPath"], filename), 'r') as file:
        infile = file.read()
    # return content as string
    return infile
```
Passing a filename as an argument to that function will load the contents of this specific file for us.
<br><br>
3. The rest of the init_pygame_and_exp() function defines some settings of our experiment: For instance, a screen surface is created, the frame rate is set, font instances are created, and the coordinates of our instructions are set.

## Specifying a `quit_pygame()` Function

Once the `init_pygame_and_exp()` function is implemented, we can also implement a function that closes PyGame and Python expicitly. This can be done at the end of our script and the function will look like this:
```python
def quit_pygame():
    """exits pygame explicitly."""
    # quit program
    pygame.quit()
    # exit python
    sys.exit()
```

## Loading Stimuli

Next, we will load all of the stimuli that are stored in a csv-file located in the directory `stimuli`. As we are programming a simplified version if a Stroop Task, our stimuli will be stored in two columns:
- The first column will hold the stimuli content
- The second column will hold the color of the stimuli
Here is the function that loads our stimuli:
```python
def load_stimuli():
    """loads stimuli lists."""

    # load items (column 0)
    results["items"] = get_items("stimuli.csv", column=0)

    # load colors (column 1)
    results["colors"] = get_items("stimuli.csv", column=1)
    # zip lists together to form list of tuples
    settings["stimlist"] = list(zip(results["items"], results["colors"]))

    # determine ground truth (congruent vs. incongruent) for each tuple in the stimlist
    [results["groundtruth"].append("congruent") if x[0] == x[1] else results["groundtruth"].append("incongruent") for x in settings["stimlist"]]
```
Note how we read in each column separately with a function `get_items()` for which the column that is to-be-read can be specified. That function is a helper function and looks like this:

```python
def get_items(filename, column):
    """
    loads items from a csv file and returns a randomly shuffled list
    that will serve as the stimuli list.
    arg1: filename
    arg2: column to read from
    return: shuffled list items
    """

    # opens the file
    with open(os.path.join(settings["stimuliPath"], filename), 'r', newline = "") as csvfile:
        # define reader
        reader = csv.reader(csvfile, delimiter=';')
        # initialize local empty list
        items = []
        # iterate over rows of sepcified column
        for row in reader:
            item = row[column]
            items.append(item) # append to local list
        # shuffle list
        random.shuffle(items)
    # return shuffled list
    return items
```
There are a couple of things that are worth mentioning about these two functions:
- In `get_items()` the column is passed as the index of each row using row[column]. You know this already from indexing lists (e.g. given a list named list1, `list1[0]` returns the first element, `list1[1]` returns the second element, etc.). Once all stimuli are appended to a list, the list is shuffled with the *in-place* operation `random.shuffle(items)`.
- `load_stimuli()`, after reading both columns separately, the elements of these two lists are zipped together to form a list of tuples with the first element of the tuple referring to the item and the second to its color (e.g. ("red", "blue")). In a final step a list `groundtruth` is defined. Here, the entry of the list is "congruent" if both elemtns of the tuples match and "incongruent" if both elements of the tuples are different. That is, we are creating a list that tells us if the stimulus that is going to be presented later is presented in the same color **(congruent)** or differing color **(incongruent)**.

## Creating Instruction Presentation Functions

Now we are finally ready to start implementing thefunctions that will actually present things to participants. Having set up the experiment correctly with the previously described functions (e.g. `init_pygame_and_exp()`, `load_stimuli()`, etc.), we are now ready to define the functions that present instructions about the experiment to participants. We will start by presenting welcome insructions:
```python
def start_welcome_block():
    """presents welcome instructions to participant."""

    # set background
    settings["screen"].fill(settings["bgColor"])
    settings["continue"] = 0

    while settings["continue"] != 1:

        # create welcome instruction object
        welcomeInst = TextPresenter.text_object(instructions["welcome"], settings["instFont"],
                                                    settings["instWidth"], settings["instHeight"])
        # blit instructions to screen
        settings["screen"].blit(welcomeInst, (settings["screenRect"].centerx - (settings["instWidth"] // 2),
                                               settings["screenRect"].centery - (settings["instHeight"] // 2)))
        # flip to foreground
        pygame.display.flip()

        # process continue event
        process_continue_event()
```
We have already learned how this function works in previous sections. Here is a quick recap: On the first line we set the background. Then we set a boolean variable `settings["continue"]` to be `False` (in pyton `0` corresponds to False and `1` to True). This variable enables is to control the while loop that follows. Inside the while-loop, the `welcomeInst` object is created using the `text_object()` function from the `TextPresenter` module and then this text object (pygame text surface to be more precise) is blitted to the screen. Finally we flip everything to the foreground and wait for a keypress. Specifically, we wait for the user to press the `RETURN` key. This is done in a separate function named `process_continue_event()`:
```python
def process_continue_event():
    """processes continue events."""
    for event in pygame.event.get():
        # handle quit event
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        elif event.type == pygame.KEYDOWN:
        # respond to a keypress
            if event.key == pygame.K_RETURN:
                settings["continue"] = 1
```
Once the `RETURN` key is pressed, our boolean varible `settings["continue"]` is set to `True` (remember in python `1` corresponds to True). This causes the while-loop to be terminated and the program moves on.
<br><br>
All other instruction presentation functions are implemented analogously. The only difference is that sometimes we are wating for a `RETURN` keypress and other times for a `SPACE` keypress. Please see the link to the entire script at the bottom of this section for the other instruction presentation implementations and the accompanying event handling functions.

## Implementing Stroop Task

Now let's turn to the heart of our experiment, the Stroop Task itself. In order to implement the simplified Stroop Task, we will implement the following functions:
- `start_task()`: controls the task by iterating over our stimuli list
- `prepare_stimulus(stimulus)`: prepares an individual stimulus for presentation by setting its color
- `create_fixation()`: creates coordinates of crossing lines for a fixation cross 
- `draw_fixation(duration)`: draws a fixation cross for `duration` of time
- `draw_stimulus()`: draws the current stimulus to screen
- `draw_isi()`: draws an empty screen as the inter-stimulus-interval (ISI)

Let's start by defining the function that iterates over our stimulus list:
```python
def start_task():
    """
    presents items in differing colors.
    arg: stimuluslist of tuples (item, color)
    """

    for stimulus in settings['stimlist']:
        # prepare red stimulus
        prepare_stimulus(stimulus)
        # draw fixcross, stimulus and ISI
        draw_fixation(1.0)
        draw_stimulus()
        draw_isi(1.0)
```
Note that all this function does is iterate over the list of tuples and inside the for loop other helper functions are called that handle the presentation and flow of each trial. The first function that is called on each trial is the `prepare_stimulus(stimulus)` function to which a stimulus (e.g. tuple containing the item and its color) is passed as an argument:
```python
def prepare_stimulus(stimulus):
    """
    Sets individual stimulus at
    center of the screen.
    arg1: stimulus to be centered
    """

    # render word depending on color
    if stimulus[1] == "red":
        color = settings["redColor"]
    else:
        color = settings['blueColor']

    # parameters are the string, anti-aliasing, color of text, color of background
    settings["item"] = settings["itemFont"].render(stimulus[0], True, color, settings["bgColor"])
    # get the rectangle of the item
    settings["itemRect"] = settings["item"].get_rect()
    # place at the center of the screen
    settings["itemRect"].center = settings["screenRect"].center
```
Note how the color is set depending on the second element of the incoming tuple (e.g. ("red", "red")). At the end of this function the item object is created in the previously determined color adn it is placed at the center of the screen. This procedure should already be familiar to you from an earlier section on how to present text stimuli.<br>
Once the stimulus is prepared for presentation, we want to present an fixation cross before the actual stimulu is drawn. This is done using the function `draw_fixation(duration)` which makes us of the helper function `create_fixation()` to get the end points of the crossing lines that comprise the fixation cross. You know these two functions already from previous sections and they look as follows:
```python
def create_fixation():
    """
    creates fixation cross by defining
    endpoints of the lines.
    """
    # Parameters are two tuples - (x1, y1) - (x2, y2)
    settings["verPoints"] = [(settings["screenRect"].centerx - settings["lineLength"]*0.5,
                       settings["screenRect"].centery),
                      (settings["screenRect"].centerx + settings["lineLength"]*0.5,
                       settings["screenRect"].centery)]

    settings["horPoints"] = [(settings["screenRect"].centerx,
                       settings["screenRect"].centery + settings["lineLength"]*0.5),
                       (settings["screenRect"].centerx,
                        settings["screenRect"].centery -settings["lineLength"]*0.5)]


def draw_fixation(duration):
    """
    draws fixation cross for duration
    of time.
    arg: duration
    """
    # create points of fixcross
    create_fixation()

    # get time stamp
    startTime = pygame.time.get_ticks() / 1000

    # set background
    settings["screen"].fill(settings["bgColor"])

    # while loop for drawing end task instructions for "duration" of time.
    while (pygame.time.get_ticks() / 1000) - startTime < duration:

        # draw vert. and hor. lines
        pygame.draw.lines(settings["screen"], settings["blackColor"], False, settings["verPoints"], settings["lineWidth"])
        pygame.draw.lines(settings["screen"], settings["blackColor"], False, settings["horPoints"], settings["lineWidth"])
        # flip to foreground
        pygame.display.flip()
        # process queue
        process_isi_event()
```
Note that we are calling an event handling function `process_isi_event()` to clear the event queue. This is necessary to avoid processing events that we are interested in before they occur. This dummy function looks as follows and it's job is to not record anything except a possible quit event:
```python
def process_isi_event():
    """processes isi event."""
    for event in pygame.event.get():
        # handle quit event
        if event.type == pygame.QUIT:
            quit_pygame()
```
Next, the `draw_stimulus()` function draws each stimulus to screen:
```python
def draw_stimulus():
    """draws stimuli to screen."""

    # get time stamp for rt recording
    t0 = pygame.time.get_ticks()
    # fill background
    settings["screen"].fill(settings["bgColor"])
    # reset response variable
    settings["response"] = None
    # while loop for drawing stimulus to screen for "duration" of time
    while settings["response"] != "congruent" and settings["response"] != "incongruent":

        # process responses
        process_response_event()
        # draw to background
        settings["screen"].blit(settings["item"], settings["itemRect"])
        # flip to foreground
        pygame.display.flip()
    # record reaction time
    rt = pygame.time.get_ticks() - t0
    # append response and rt to results dicts
    results["rts"].append(rt)
    results["responses"].append(settings["response"])
```
Note that we are waiting for one of the following two response events by the user:
- F-key if item and color match (congruent)
- J-key if item and color mismatch (incongruent)
The event handler that processes these events looks as follows:
```python
def process_response_event():
    """processes response event."""
    for event in pygame.event.get():
        # handle quit event
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        elif event.type == pygame.KEYDOWN:
        # respond to a keypress
            if event.key == pygame.K_f:
                settings["response"] = "congruent"
            elif event.key == pygame.K_j:
                settings["response"] = "incongruent"
```
Also note how a timestamp `t0` is created inside of `draw_stimulus()` before the while-loop starts. This timestamp is then later used to calculate the reaction time once an event has occured and the while-loop has been exited. At this time the response and reaction time are also appended to the results dictionary using the following lines of code:
```python
# record reaction time
rt = pygame.time.get_ticks() - t0
# append response and rt to results dicts
results["rts"].append(rt)
results["responses"].append(settings["response"])
```
Finally, the `draw_isi()` function  draws a blank screen at the end of each trial:
```python
def draw_isi(duration):
    """
    draws blank inter-stimulus-interval
    for duration of time.
    arg: duration
    """

    # set background
    settings["screen"].fill(settings["bgColor"])

    # get time stamp
    startTime = pygame.time.get_ticks() / 1000

    # while loop for drawing ISI
    while (pygame.time.get_ticks() / 1000) - startTime < duration:

        # just flip empty screen to foreground
        pygame.display.flip()
        # process event in isi
        process_isi_event()
```

## Saving Results

Before the experiment is closed, we need to save the contents of our `results` dictionary to a csv-file. This is done with a function that should be familiar to you from previous sections. Here is the code:
```python
def save_results(filename, resultsdict):
    """
    saves results to a csv file.
    arg1: filename
    arg2: dictionary holding resultsdict
    """
    # open data file
    with open(os.path.join(settings["dataPath"], filename), 'w', newline="") as file:
        # create csv writer
        w = csv.writer(file, delimiter=';')
        # write first row (variable labels)
        w.writerow(resultsdict.keys())
        # write data row wise
        w.writerows(zip_longest(*resultsdict.values()))
```
Inside this function we make use of the `zip_longest()` function to write each element of the lists row wise to the csv-file. To find out more about this function and the `itertools` module read the following docs:
<br><br>
**[itertools](https://docs.python.org/3.5/library/itertools.html)**


## Complete Stroop Experiment
The complete code for the experiment can be found here:<br><br>
**[stroop.py](https://github.com/imarevic/PsyPythonCourse/blob/master/notebooks/Chapter9/stroop.py)**
<br><br>
Study the code carefully and try to understand what each function within the `run_experiment()` function does. A good approach is to uncomment all functions except the first one and run the program to see what happens. Then repeat that process iteratively with the other fucntions until you understand what each function and its helper functions do. In doing so, keep in mind that some functions depend on others, so uncommenting one might cause errors when running another function.